In [1]:
from dependencies import *
from defaults import *
from model import *
from MC_helpers import *
from save_data import *
from background import *

### Find the parameter space to use

We want to simulate emplacement of other LIPs *only* in the parameter space where the Franklin causes a Snowball. This retrieves that parameter space.

In [2]:
## read in the Franklin data

# number of runs to analyze
iters = 100_000
iter_str = str(iters) if iters < 1000 else f"{int(iters/1000)}k"

# read in file and get parameters
filename = f"data/Franklin_{iter_str}.nc"

with Dataset(filename, 'r') as ds:
        params_group = ds.groups['params']
        snowball_flag = ds.groups['stats'].variables['snowball_flag'][:]
        params = {var_name: var[:][snowball_flag == 1] for var_name, var in params_group.variables.items()}

# return variables to correct units for input into model
params["A0"] *= 1e12 # Mkm2 to m
params["B0"] *= 1e3 # km to m
        
# get rid of constructed variables       
del params["E0"]
del params["erup_num"]

iters = len(params['dt'])
print(f"Generated parameter dictionary for {iters} runs")

# save these parameters
params_baseline = params.copy()

Generated parameter dictionary for 9363 runs


### Franklin generation

Generates datasets for the Franklin LIP with minimum and maximum area estimates. Since the areas of Phanerozoic LIPs are well-characterized and therefore have a single value, this allows for a more direct comparison of the probability distributions.

In [3]:
# Large Franklin
A0_max = np.max(param_ranges['A0'])
print(f"Running with area = {A0_max/1e12:0.0f} Mkm\u00b2")

params = params_baseline.copy()
params['A0'] = [A0_max]*iters

t,T = run_model_iters_parallel(params)
filename = f"data/Franklin_large_filtered.nc"
save_data(filename,t,T,params,
         description="Franklin with area fixed to max estimate")
print("")
summary_stats(filename)

Running with area = 11 Mkm²
Starting 9,363 iterations at 06:04 PM
Estimating completion in 6 minutes at 06:11 PM
    Completed 520/9,363 iterations after 30 seconds
    Completed 1,064/9,363 iterations after 1 minutes
    Completed 1,562/9,363 iterations after 1 minutes
    Completed 2,083/9,363 iterations after 2 minutes
    Completed 2,593/9,363 iterations after 2 minutes
    Completed 3,111/9,363 iterations after 3 minutes
    Completed 3,640/9,363 iterations after 3 minutes
    Completed 4,180/9,363 iterations after 4 minutes
    Completed 4,726/9,363 iterations after 4 minutes
    Completed 5,223/9,363 iterations after 5 minutes
    Completed 5,767/9,363 iterations after 5 minutes
    Completed 6,308/9,363 iterations after 6 minutes


/Users/cminsky/Library/CloudStorage/OneDrive-HarvardUniversity/Res_Franklin/new_model/model.py:89: RuntimeWarning: invalid value encountered in scalar multiply
  P_i *= y**n_p # climate dependence


    Completed 6,851/9,363 iterations after 6 minutes
    Completed 7,395/9,363 iterations after 7 minutes
    Completed 7,941/9,363 iterations after 7 minutes
    Completed 8,466/9,363 iterations after 8 minutes
    Completed 8,985/9,363 iterations after 8 minutes
Completed 9,363 iterations after 8 minutes at 06:13 PM
Data saved to data/Franklin_large.nc

Average cooling: -13.37 K
Late flag percentage: 100.00%
Early flag percentage: 66.52%
Snowball flag percentage: 33.48%


In [3]:
# Small Franklin
A0_min = np.min(param_ranges['A0'])
print(f"Running with area = {A0_min/1e12:0.0f} Mkm\u00b2")

params = params_baseline.copy()
params['A0'] = [A0_min]*iters

t,T = run_model_iters_parallel(params)
filename = f"data/Franklin_small_filtered.nc"
save_data(filename,t,T,params,
         description="Franklin with area fixed to min estimate")
print("")
summary_stats(filename)

Running with area = 3 Mkm²
Starting 9,363 iterations at 08:43 PM
Estimating completion in 6 minutes at 08:49 PM
    Completed 315/9,363 iterations after 30 seconds
    Completed 626/9,363 iterations after 1 minutes
    Completed 970/9,363 iterations after 1 minutes
    Completed 1,303/9,363 iterations after 2 minutes
    Completed 1,663/9,363 iterations after 2 minutes
    Completed 2,028/9,363 iterations after 3 minutes
    Completed 2,388/9,363 iterations after 3 minutes
    Completed 2,737/9,363 iterations after 4 minutes
    Completed 3,093/9,363 iterations after 4 minutes
    Completed 3,442/9,363 iterations after 5 minutes
    Completed 3,800/9,363 iterations after 5 minutes
    Completed 4,159/9,363 iterations after 6 minutes
    Completed 4,529/9,363 iterations after 6 minutes
    Completed 4,893/9,363 iterations after 7 minutes
    Completed 5,232/9,363 iterations after 7 minutes
    Completed 5,544/9,363 iterations after 8 minutes
    Completed 5,832/9,363 iterations after 8 

### Phanerozoic LIP generation

Runs simulations for Phanerozoic LIPs in the same parameter space, with background climate and LIP area specific to each LIP.

In [4]:
def generate_LIP_filtered(LIP_name,t_Earth,A0,
                          iters=iters,params_baseline=params_baseline,
                          print_progress=True):
    # get background climate
    N0,V,T0 = background_ranges(t_Earth,iters)
    if t_Earth <= 419:
        N0 = Foster_CO2(t_Earth,iters)
    if t_Earth <= 540:
        T0 = Scotese_T(t_Earth,iters)
    
    # print average background climate conditions
    if print_progress:
        CO2 = 280*(N0/2.83)**2
        print(f"Mean pCO2: {np.mean(CO2):0.0f} ppm, T: {np.mean(T0):0.0f} K")

    # update parameter set
    params = params_baseline.copy()
    params['N0'] = N0
    params['V'] = V
    params['T0'] = T0
    params['A0'] = [A0] * iters # LIP area [m2]
    
    # run model
    t,T = run_model_iters_parallel(params)
    filename = f"data/{LIP_name}_filtered.nc"
    save_data(filename,t,T,params,print_progress=print_progress,
             description="Standard CAMP run with degassing fixed at 3.73 examol CO2 and erosion sampled randomly.")
    
    # print summary statistics
    if print_progress:
        print("")
        summary_stats(filename)

In [7]:
generate_LIP_filtered(LIP_name="CAMP", # LIP name
                      t_Earth = 201, # age [Mya]
                      A0 = 11.46e12) # LIP area [m2]

Mean pCO2: 1717 ppm, T: 295 K
Starting 9,363 iterations at 08:00 PM
Estimating completion in 6 minutes at 08:07 PM
    Completed 401/9,363 iterations after 30 seconds
    Completed 782/9,363 iterations after 1 minutes
    Completed 1,154/9,363 iterations after 1 minutes
    Completed 1,548/9,363 iterations after 2 minutes
    Completed 1,972/9,363 iterations after 2 minutes
    Completed 2,391/9,363 iterations after 3 minutes
    Completed 2,821/9,363 iterations after 3 minutes
    Completed 3,241/9,363 iterations after 4 minutes
    Completed 3,666/9,363 iterations after 4 minutes
    Completed 4,085/9,363 iterations after 5 minutes
    Completed 4,503/9,363 iterations after 5 minutes
    Completed 4,911/9,363 iterations after 6 minutes
    Completed 5,332/9,363 iterations after 6 minutes
    Completed 5,733/9,363 iterations after 7 minutes
    Completed 6,139/9,363 iterations after 7 minutes
    Completed 6,575/9,363 iterations after 8 minutes


/Users/cminsky/Library/CloudStorage/OneDrive-HarvardUniversity/Res_Franklin/new_model/model.py:89: RuntimeWarning: invalid value encountered in scalar multiply
  P_i *= y**n_p # climate dependence


    Completed 7,023/9,363 iterations after 8 minutes
    Completed 7,472/9,363 iterations after 9 minutes
    Completed 7,926/9,363 iterations after 9 minutes
    Completed 8,372/9,363 iterations after 10 minutes
    Completed 8,804/9,363 iterations after 10 minutes
    Completed 9,216/9,363 iterations after 11 minutes
Completed 9,363 iterations after 11 minutes at 08:12 PM
Saving results...
Saving parameters...
Data saved to data/CAMP_filtered.nc

Average cooling: -14.04 K
Late flag percentage: 30.47%
Early flag percentage: 18.62%
Snowball flag percentage: 11.86%


In [6]:
generate_LIP_filtered(LIP_name="Kola-Dnieper", # LIP name
                      t_Earth = 380, # age [Mya]
                      A0 = 5.9e12) # LIP area [m2]

/Users/cminsky/Library/CloudStorage/OneDrive-HarvardUniversity/Res_Franklin/new_model/background.py:68: RuntimeWarning: invalid value encountered in sqrt
  N0 = N_pi*np.sqrt(CO2/ppm_pi) # surficial carbon [examol]


Mean pCO2: nan ppm, T: 292 K
Starting 9,363 iterations at 07:49 PM
Estimating completion in 6 minutes at 07:56 PM
    Completed 403/9,363 iterations after 30 seconds
    Completed 805/9,363 iterations after 1 minutes
    Completed 1,185/9,363 iterations after 1 minutes
    Completed 1,584/9,363 iterations after 2 minutes
    Completed 2,018/9,363 iterations after 2 minutes
    Completed 2,472/9,363 iterations after 3 minutes
    Completed 2,899/9,363 iterations after 3 minutes
    Completed 3,333/9,363 iterations after 4 minutes
    Completed 3,769/9,363 iterations after 4 minutes
    Completed 4,216/9,363 iterations after 5 minutes
    Completed 4,662/9,363 iterations after 5 minutes
    Completed 5,094/9,363 iterations after 6 minutes
    Completed 5,524/9,363 iterations after 6 minutes
    Completed 5,925/9,363 iterations after 7 minutes
    Completed 6,338/9,363 iterations after 7 minutes
    Completed 6,777/9,363 iterations after 8 minutes
    Completed 7,221/9,363 iterations afte

/Users/cminsky/Library/CloudStorage/OneDrive-HarvardUniversity/Res_Franklin/new_model/model.py:89: RuntimeWarning: invalid value encountered in scalar multiply
  P_i *= y**n_p # climate dependence


    Completed 8,898/9,363 iterations after 10 minutes
    Completed 9,329/9,363 iterations after 11 minutes
Completed 9,363 iterations after 11 minutes at 08:00 PM
Saving results...
Saving parameters...
Data saved to data/Kola-Dnieper_filtered.nc

Average cooling: -9.17 K
Late flag percentage: 20.60%
Early flag percentage: 6.39%
Snowball flag percentage: 14.22%


In [5]:
generate_LIP_filtered(LIP_name="Kalkarindji", # LIP name
                      t_Earth = 511, # age [Mya]
                      A0 = 3.54e12) # LIP area [m2]

Mean pCO2: 938 ppm, T: 298 K
Starting 9,363 iterations at 07:38 PM
Estimating completion in 6 minutes at 07:45 PM
    Completed 522/9,363 iterations after 30 seconds
    Completed 1,036/9,363 iterations after 1 minutes
    Completed 1,559/9,363 iterations after 1 minutes
    Completed 2,053/9,363 iterations after 2 minutes
    Completed 2,529/9,363 iterations after 2 minutes
    Completed 2,930/9,363 iterations after 3 minutes
    Completed 3,394/9,363 iterations after 3 minutes
    Completed 3,862/9,363 iterations after 4 minutes
    Completed 4,211/9,363 iterations after 4 minutes
    Completed 4,584/9,363 iterations after 5 minutes
    Completed 4,974/9,363 iterations after 5 minutes
    Completed 5,333/9,363 iterations after 6 minutes
    Completed 5,723/9,363 iterations after 6 minutes
    Completed 6,130/9,363 iterations after 7 minutes
    Completed 6,494/9,363 iterations after 7 minutes
    Completed 6,919/9,363 iterations after 8 minutes
    Completed 7,353/9,363 iterations af